Exercise 8


In [8]:
from sklearn.datasets import fetch_openml

In [9]:
data = fetch_openml(name="mnist_784", version = "active")


In [10]:
X = data["data"]
y = data["target"]

In [11]:
#split instances for training and test/val
from sklearn.model_selection import train_test_split
X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=20000)

In [12]:
#Split instances for test and validation sets
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size = 10000)

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC


In [14]:
#Create our models
Logistic_clf = LogisticRegression()
forest_clf = RandomForestClassifier(n_estimators=150, max_depth=5, min_samples_split=100)
Extra_tree = ExtraTreesClassifier(n_estimators=150, max_depth=5, min_samples_split=100)
svc = SVC(probability=True)
linear_svc = LinearSVC()


In [15]:
#Logistic_clf.fit(X_train,y_train)
forest_clf.fit(X_train,y_train)
Extra_tree.fit(X_train,y_train)
svc.fit(X_train,y_train)
#linear_svc.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [16]:
forest_predictions = forest_clf.predict(X_val)
Extra_tree_predictions = Extra_tree.predict(X_val)
svc_predictions = svc.predict(X_val)

In [31]:
from sklearn.metrics import accuracy_score
print(f"Accuracy for forest: {accuracy_score(y_val,forest_predictions)}")
print(f"Accuracy for Extra-trees: {accuracy_score(y_val,Extra_tree_predictions)}")
print(f"Accuracy for svc: {accuracy_score(y_val,svc_predictions)}")

Accuracy for forest: 0.8554
Accuracy for Extra-trees: 0.8291
Accuracy for svc: 0.8634


In [32]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(estimators=[("forest", forest_clf), ("Extra-trees",Extra_tree),("SVC" ,svc)],
                              voting='hard'
                              )

In [33]:
voting_clf.fit(X_train,y_train)

ValueError: ignored

In [ ]:
voting_predictions = voting_clf.predict(X_val)


In [ ]:
voting_clf.score(X_val, y_val)

In [ ]:
print(f"Accuracy for hard voting: {accuracy_score(y_val,voting_predictions)}")

In [ ]:
#Try to change to soft voting
voting_clf.voting = "soft"
voting_clf.score(X_val, y_val)

# **Exercise 9**
## * Let's Create a Blender!


In [26]:
# Let's save our trained models

import joblib
from google.colab import files
def model_saver_and_downloader(model, name):
  joblib.dump(model, f"{name}.pkl")
  files.download(f"{name}.pkl")

In [38]:

model_saver_and_downloader(forest_clf, "forest_clf")
model_saver_and_downloader(Extra_tree, "Extra_tree")
model_saver_and_downloader(svc, "svc")
model_saver_and_downloader(voting_clf, "voting_clf")

#Calling the model and using it
#model_saved = joblib.load("model_name") this calls our trained model


#Let's Create a Blender!
logistic_clf_blender = LogisticRegression()
import pandas as pd


#Create a new training set with the validation prediction of all the predictors
first_layer_predictions_training_set =  pd.DataFrame()
first_layer_predictions_training_set['forest_predictions'] = forest_predictions
first_layer_predictions_training_set['Extra_trees_predictions'] = Extra_tree_predictions
first_layer_predictions_training_set['SVC_predictions'] = linear_svc_predictions
first_layer_predictions_training_set['labels'] = y_val



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [71]:
#Split the new dataset into X and y
X_blender = first_layer_predictions_training_set.to_numpy()[:, :3]
y_blender = first_layer_predictions_training_set['labels']
logistic_clf_blender.fit(X_blender,y_blender)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [72]:
#Test the blender with the test_set
#create predictions for the test test and create a new test set for predictions 


blender_test_set  =  pd.DataFrame()
blender_test_set['forest_predictions'] = forest_test_predictions
blender_test_set['Extra_trees_predictions'] = Extra_tree_test_predictions
blender_test_set['SVC_predictions'] = svc_test_predictions


blender_predictions = logistic_clf_blender.predict(blender_test_set)


#Check accuracy
accuracy_score(y_test, blender_predictions)

ValueError: ignored